# CMA-ES


**Disclaimer:** We make use of the implementation available at [PyPi](https://pypi.org/project/cma/) <cite data-cite="pycma"></cite> published by the author Nikolaus Hansen under the BSD license. 


CMA-ES which was proposed in <cite data-cite="cmaes"></cite>. Moreover, a comparing review can be found in <cite data-cite="cmaes-review"></cite>. 
CMA-ES stands for covariance matrix adaptation evolution strategy. Evolution strategies (ES) are stochastic, derivative-free methods for numerical optimization of non-linear or non-convex continuous optimization problems. They belong to the class of evolutionary algorithms and evolutionary computation. An evolutionary algorithm is broadly based on the principle of biological evolution, namely the repeated interplay of variation (via recombination and mutation) and selection: in each generation (iteration) new individuals (candidate solutions) are generated by variation, usually in a stochastic way, of the current parental individuals. Then, some individuals are selected to become the parents in the next generation based on their fitness or objective function value 
$f(x)$. Like this, over the generation sequence, individuals with better and better $f$-values are generated.
(excerpt from [Wikipedia](https://en.wikipedia.org/wiki/CMA-ES)).

### Example

In [1]:
import numpy as np

from pymoo.algorithms.soo.nonconvex.cmaes import CMAES
from pymoo.factory import get_problem
from pymoo.optimize import minimize

problem = get_problem("sphere")

algorithm = CMAES(x0=np.random.random(problem.n_var))

res = minimize(problem,
               algorithm,
               seed=1,
               verbose=False)

print(f"Best solution found: \nX = {res.X}\nF = {res.F}\nCV= {res.CV}")



Compiled modules for significant speedup can not be used!
https://pymoo.org/installation.html#installation

To disable this warning:
from pymoo.config import Config
Config.show_compile_hint = False



Best solution found: 
X = [0.49999997 0.49999999 0.49999996 0.49999996 0.49999995 0.49999998
 0.49999999 0.50000004 0.50000003 0.5       ]
F = [9.58652921e-15]
CV= [0.]


CMA-ES already has several stopping criteria implemented. However, as for other algorithms, the number of iterations or function evaluations can be directly passed to `minimize`.

In [2]:
res = minimize(problem,
               algorithm,
               ('n_iter', 10),
               seed=1,
               verbose=True)

print("Best solution found: \nX = %s\nF = %s" % (res.X, res.F))

n_gen |  n_eval |     fopt     |    sigma     | min std  | max std  |   axis  
    1 |       1 |  0.714363775 |  0.100000000 |  0.10000 |  0.10000 |  1.00005
    2 |      11 |  0.453453996 |  0.098052363 |  0.09618 |  0.10032 |  1.14881
    3 |      21 |  0.385123840 |  0.112253571 |  0.10925 |  0.11776 |  1.23856
    4 |      31 |  0.272450662 |  0.130945976 |  0.12695 |  0.13890 |  1.32272
    5 |      41 |  0.272450662 |  0.140262858 |  0.13670 |  0.14703 |  1.30914
    6 |      51 |  0.124930775 |  0.153295809 |  0.14531 |  0.15875 |  1.28965
    7 |      61 |  0.059011320 |  0.163766172 |  0.15519 |  0.16958 |  1.28348
    8 |      71 |  0.059011320 |  0.170831217 |  0.16002 |  0.17659 |  1.39014
    9 |      81 |  0.059011320 |  0.166423481 |  0.15488 |  0.17127 |  1.41629
   10 |      91 |  0.059011320 |  0.155138351 |  0.14047 |  0.15717 |  1.46693
Best solution found: 
X = [0.45182261 0.58711369 0.524771   0.54063447 0.41602516 0.39631345
 0.4058968  0.47565089 0.59797183 0.40

In [3]:
res = minimize(problem,
               algorithm,
               ('n_evals', 50),
               seed=1,
               verbose=True)

print("Best solution found: \nX = %s\nF = %s" % (res.X, res.F))

n_gen |  n_eval |     fopt     |    sigma     | min std  | max std  |   axis  
    1 |       1 |  0.714363775 |  0.100000000 |  0.10000 |  0.10000 |  1.00005
    2 |      11 |  0.478126421 |  0.098800626 |  0.09431 |  0.10636 |  1.23503
    3 |      21 |  0.347947441 |  0.111882679 |  0.10653 |  0.12153 |  1.33616
    4 |      31 |  0.238447238 |  0.123797099 |  0.12036 |  0.14047 |  1.42252
    5 |      41 |  0.180754921 |  0.143353120 |  0.13776 |  0.16946 |  1.54241
    6 |      51 |  0.131369818 |  0.154594810 |  0.14821 |  0.18189 |  1.56638
Best solution found: 
X = [0.46835763 0.36873371 0.4350829  0.43362848 0.34325381 0.40883132
 0.38858676 0.51690256 0.38268162 0.71254417]
F = [0.13136982]


Also, easily restarts can be used, which are known to work very well on multi-modal functions. For instance, `Rastrigin` can be solved rather quickly by:

In [4]:
problem = get_problem("rastrigin")

algorithm = CMAES(restarts=10, restart_from_best=True)

res = minimize(problem,
               algorithm,
               ('n_evals', 2500),
               seed=1,
               verbose=False)

print("Best solution found: \nX = %s\nF = %s" % (res.X, res.F))

Best solution found: 
X = [ 1.18439125e-09 -2.76736767e-09]
F = [0.]


Our framework internally calls the `cma.fmin2` function. All parameters that can be used there either as a keyword argument or an option can also be passed to the `CMAES` constructor.
An example with a few selected `cma.fmin2` parameters is shown below:

In [5]:
import numpy as np
from pymoo.util.normalization import denormalize

# define an intitial point for the search
np.random.seed(1)
x0 = denormalize(np.random.random(problem.n_var), problem.xl, problem.xu)

algorithm = CMAES(x0=x0,
                 sigma=0.5,
                 restarts=2,
                 maxfevals=np.inf,
                 tolfun=1e-6,
                 tolx=1e-6,
                 restart_from_best=True,
                 bipop=True)

res = minimize(problem,
               algorithm,
               seed=1,
               verbose=False)

print("Best solution found: \nX = %s\nF = %s" % (res.X, res.F))

Best solution found: 
X = [9.13729185e-07 1.40284031e-06]
F = [5.56067192e-10]


For more details about hyperparameters, we refer to the software documentation of the `fmin2` in CMA-ES, which can be found [here](http://cma.gforge.inria.fr/apidocs-pycma/cma.evolution_strategy.html#fmin2).
A quick explanation of possible parameters is also provided in the API documentation below.

### API